In [1]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm
from pathlib import Path
from multiprocessing.pool import ThreadPool

In [2]:
files = Path('data').glob('*.pkl')

In [3]:
datas = pd.concat(list(map(lambda x: pd.read_pickle(x), files)))

In [4]:
datas

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-1INCH,2021-10-27T08:59:00,2021-10-27T17:59:00,10550.0,10750.0,10340.0,10700.0,1635325200277,1.169192e+10,1.111414e+06,1
1,KRW-1INCH,2021-10-27T08:58:00,2021-10-27T17:58:00,10840.0,11090.0,10530.0,10570.0,1635325141041,1.200504e+10,1.108238e+06,1
2,KRW-1INCH,2021-10-27T08:57:00,2021-10-27T17:57:00,11410.0,11630.0,10710.0,10850.0,1635325080776,1.431111e+10,1.290275e+06,1
3,KRW-1INCH,2021-10-27T08:56:00,2021-10-27T17:56:00,11690.0,12190.0,11400.0,11450.0,1635325020268,1.054660e+10,8.858417e+05,1
4,KRW-1INCH,2021-10-27T08:55:00,2021-10-27T17:55:00,11330.0,11760.0,11200.0,11690.0,1635324960120,9.415980e+09,8.154110e+05,1
...,...,...,...,...,...,...,...,...,...,...,...
63,KRW-ZRX,2022-03-16T15:06:00,2022-03-17T00:06:00,618.0,619.0,618.0,619.0,1647443213343,6.709732e+04,1.085553e+02,1
64,KRW-ZRX,2022-03-16T15:04:00,2022-03-17T00:04:00,618.0,618.0,618.0,618.0,1647443077161,4.118493e+05,6.664229e+02,1
65,KRW-ZRX,2022-03-16T15:03:00,2022-03-17T00:03:00,618.0,618.0,618.0,618.0,1647443022594,6.287000e+03,1.017314e+01,1
66,KRW-ZRX,2022-03-16T15:01:00,2022-03-17T00:01:00,617.0,618.0,617.0,617.0,1647442902835,2.311694e+06,3.746438e+03,1


In [5]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55159364 entries, 0 to 67
Data columns (total 11 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   market                   object 
 1   candle_date_time_utc     object 
 2   candle_date_time_kst     object 
 3   opening_price            float64
 4   high_price               float64
 5   low_price                float64
 6   trade_price              float64
 7   timestamp                int64  
 8   candle_acc_trade_price   float64
 9   candle_acc_trade_volume  float64
 10  unit                     int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 4.9+ GB


In [6]:
datas_dict = datas.to_dict('records')

In [7]:
session = requests.Session()
retry = Retry(connect=1000, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [8]:
def start(data):
    insert_url = 'http://127.0.0.1:8000/upbit/candle/'
    session.post(insert_url, data=data)

In [ ]:
pool = ThreadPool(processes=100)
pool.map(start, tqdm(datas))
pool.close()
pool.join()

  0%|          | 11/55159364 [00:00<322:04:50, 47.57it/s]
